In [2]:
with open('test.log', 'r') as f:
    for line in f:
        print(line, end='')

{0: [{'ASD': [PosixPath('/workspace/data/segmentation_dataset_512/json/ASD/20170313_ASD_lat_V1-0001.json'), PosixPath('/workspace/data/segmentation_dataset_512/json/ASD/20170313_ASD_lat_V1-0002.json'), PosixPath('/workspace/data/segmentation_dataset_512/json/ASD/20170328_ASD_lat_V1-0001.json'), PosixPath('/workspace/data/segmentation_dataset_512/json/ASD/20170328_ASD_lat_V1-0002.json'), PosixPath('/workspace/data/segmentation_dataset_512/json/ASD/20170522_1_ASD_lat_V1-0001.json'), PosixPath('/workspace/data/segmentation_dataset_512/json/ASD/20170522_1_ASD_lat_V1-0002.json'), PosixPath('/workspace/data/segmentation_dataset_512/json/ASD/20170522_1_ASD_lat_V1-0003.json'), PosixPath('/workspace/data/segmentation_dataset_512/json/ASD/20170522_1_ASD_lat_V1-0004.json'), PosixPath('/workspace/data/segmentation_dataset_512/json/ASD/20170522_1_ASD_lat_V1-0005.json'), PosixPath('/workspace/data/segmentation_dataset_512/json/ASD/20170619_1_ASD_lat__V1-0001.json'), PosixPath('/workspace/data/segmen

# Draw the histogram of the data

# Need to draw the data distribution of the dataset